In [3]:
import build_vocabulary as bv
import features_extraction as fe
voc = bv.get_vocabulary(path='../dataset/', numWords=5000, save=False)
X_train, Y_train = fe.get_bow_representation(voc)
X_valid, Y_valid = fe.get_bow_representation(voc, no_clickbait_path='../dataset/non_clickbait_validation.txt',
                                             clickbait_path='../dataset/clickbait_validation.txt', save=False, stemming=False)

print(voc)

['aaron', 'abandoned', 'abc', 'abducted', 'able', 'aboard', 'abortion', 'about', 'abroad', 'absolute', 'absolutely', 'abu', 'abuse', 'abused', 'academy', 'accept', 'accepts', 'access', 'accessories', 'accident', 'accidentally', 'accidents', 'according', 'account', 'accounts', 'accurate', 'accurately', 'accusations', 'accused', 'accuses', 'aclu', 'acne', 'acquitted', 'across', 'act', 'action', 'activist', 'activists', 'activities', 'activity', 'actor', 'actors', 'actress', 'acts', 'actual', 'actually', 'adam', 'add', 'added', 'addict', 'addicted', 'addiction', 'addicts', 'address', 'addresses', 'adds', 'adele', 'adhd', 'administration', 'admit', 'admits', 'adopt', 'adopts', 'adorable', 'adorably', 'ads', 'adult', 'adults', 'advance', 'advances', 'advantage', 'advent', 'adventure', 'advertising', 'advice', 'adviser', 'advocates', 'affair', 'affairs', 'affect', 'affected', 'affects', 'affordable', 'afghan', 'afghanistan', 'afghans', 'africa', 'african', 'after', 'aftershock', 'again', 'ag

### Training a Naive Bayes classifier

In [4]:
import pvml

probs, prior = pvml.categorical_naive_bayes_train(X_train, Y_train)
predictions_train, score= pvml.categorical_naive_bayes_inference(
    X_train, probs, prior)
accuracy_train=(predictions_train==Y_train).mean()

predictions_validation, score = pvml.categorical_naive_bayes_inference(
    X_valid, probs, prior)
accuracy_validation=(predictions_validation==Y_valid).mean()
print('Train accuracy: ',accuracy_train*100)
print('Validation accuracy: ',accuracy_validation*100)

Train accuracy:  97.6
Validation accuracy:  96.575


### Training logistic regression